## **Imports**

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt

from tabulate import tabulate
from IPython.display import display, HTML,clear_output
from tqdm import tqdm

import sys
sys.path.append("F:\Austin\helpers")
from helpers.experiment_v3 import *
#from openfiles import *

# Importing Instrument Drivers
from pymeasure.instruments.srs import sr830, sr844
from pymeasure.instruments.yokogawa import yokogawa7651, yokogawaGS200
from pymeasure.instruments.agilent import Agilent33500
#from newinstruments.SignalHound import SignalHoundSA124B
from newinstruments.SignalCore import SignalCore
from newinstruments.BlueFors import BlueFors
#from newinstruments.HP8648B import *
from newinstruments.mcc_daq import *
from newinstruments.bncRF import *
from newinstruments.vna_E5071_2 import *
from newinstruments.DPG202 import *

# Assigning Yokogawa GPIB Addresses
yoko_lch    = yokogawaGS200.YokogawaGS200("GPIB0::7::INSTR")  # Left channel electrode
yoko_rgd    = yokogawaGS200.YokogawaGS200("GPIB0::1::INSTR")  # Right channel, left gate electrode (guard)
yoko_lgt    = yokogawaGS200.YokogawaGS200("GPIB0::21::INSTR") # Left channel, left gate electrode
yoko_rgt    = yokogawaGS200.YokogawaGS200("GPIB0::6::INSTR")  # Left channel, right gate electrode
yoko_lres   = yokogawa7651.Yokogawa7651("GPIB0::24::INSTR")   # Left reservoir electrode dc component
yoko_mres   = yokogawa7651.Yokogawa7651("GPIB0::25::INSTR")   # Middle reservoir electrode dc component
# Assigning Lock-in Amplifier GPIB Addresses
lockin_LF   = sr830.SR830('GPIB0::10::INSTR')
lockin_HF   = sr844.SR844('GPIB0::11::INSTR')
# Assigning Agilent Source GPIB Addresses
gen_sign    = Agilent33500("GPIB::19::INSTR")
gen_fila    = Agilent33500("GPIB::17::INSTR")
# Assigning VNA GPIB Address (if connected)
vna         = E5071_2('GPIB0::2::INSTR')
# Assigning Pressure Gauge COM Port
#dpg202     = DPG202('COM4')

# Unused Devices 
#bnc         = BNC845()
#gen_high    = HP8648B("GPIB::20::INSTR")
#sigcore     = SignalCore(name="SignalCore", address="10003410")

# Fridge Temperature Sensors
bluefors    = BlueFors()

# Print connection errors if printing=True
connect_instruments(printing=False)
# Print the table of instrument statuses
show_connection_table(instrument_status);

Instrument,Connected,Address
VNA,✔,GPIB0::2::INSTR
SR830,✔,GPIB0::10::INSTR
SR844,✔,GPIB0::11::INSTR
DPG202,✘,COM4
Yoko (lch),✔,GPIB0::7::INSTR
Yoko (rgd),✔,GPIB0::1::INSTR
Yoko (lgt),✔,GPIB0::21::INSTR
Yoko (rgt),✔,GPIB0::6::INSTR
Yoko (lres),✔,GPIB0::24::INSTR
Yoko (mres),✔,GPIB0::25::INSTR


## **Dictionaries**

In [2]:
# Dictionaries for control of instrument functions
control_instr_dict = {
        # Sweepable Parameters
                'Vlch': [   0,  yoko_lch, 'source_voltage',   'ramp_to_voltage',   'V'],
                'Vlgt': [   0,  yoko_lgt, 'source_voltage',   'ramp_to_voltage',   'V'],
                'Vrgt': [   0,  yoko_rgt, 'source_voltage',   'ramp_to_voltage',   'V'],
               'Vsign': [0.01,  gen_sign,      'amplitude', 'ramp_to_amplitude', 'Vpp'],
               'Fsign': [ 3e6,  gen_sign,      'frequency', 'ramp_to_frequency',  'Hz'],
        # Constant Parameters
                'Vrgd': [   0,  yoko_rgd, 'source_voltage',   'ramp_to_voltage',   'V'],
               'Vlres': [   0, yoko_lres, 'source_voltage',   'ramp_to_voltage',   'V'],
               'Vmres': [   0, yoko_lres, 'source_voltage',   'ramp_to_voltage',   'V'],
               'Vfila': [0.01,  gen_fila,      'amplitude', 'ramp_to_amplitude', 'Vpp'],
               'Ffila': [ 1e1,  gen_fila,      'frequency', 'ramp_to_frequency',  'Hz'],
#             'hf_pow': [    -25,   sc,         'set_power',    'ramp_to_power',  'dB'],
#            'hf_freq': [4.0*GHz,   sc,     'set_frequency','ramp_to_frequency',  'Hz'],
#            'sa_freq': [  4*GHz,  sad,'set_freq_for_power','ramp_to_frequency',  'Hz']
}

# Dictionaries for control and readout of the VNA
vna_control_dict = {
        # Sweepable Parameters
          'freq_range': [      (3.00e9, 6.00e9), vna,  'set_frequency_range',  'Hz'],
               'power': [                   -30, vna,            'set_power', 'dBm'],
          'elec_delay': [                 10e-9, vna, 'set_electrical_delay',   's'],
        # Constant Parameters 
        'set_vna_meas': [    ('meas', 'S21', 1), vna,      'set_measure_def',    ''],
              'format': [                'MLOG', vna,           'set_format',    ''], 
              'output': [                 False, vna,           'set_output',    ''], 
           'sweep_pts': [                 10001, vna,     'set_sweep_points', 'pts'],  
          'getf_range': [      (3.00e9, 6.00e9), vna,  'get_frequency_range',  'Hz'],
          'set_bwidth': [                   1e3, vna,             'set_ifbw',  'Hz'],
             'num_avg': [                     1, vna,         'set_averages','avgs'],
           'avg_state': [                 'OFF', vna,    'set_average_state',    '']
}

# Dictionaries for readout of instrument values
readout_instr_dict = {
                'Vlfx': [lockin_LF,           'x',  'V'],
                'Vlfy': [lockin_LF,           'y',  'V'],
                'Vhfx': [lockin_HF,           'x',  'V'],
                'Vhfy': [lockin_HF,           'y',  'V'],
            'vna_meas': [      vna, 'read_data_y', 'dB'],
            'vna_freq': [      vna, 'get_fpoints', 'Hz']
#             'sa_pow': [      sad, 'get_power_at_freq', 'dB']
}

# Dictionaries for readout of BlueFors values
readout_bluefors_dict = {
               'T_50K': [ bluefors, lambda bf: bf.get_temperature(1), 'K'],
                'T_4K': [ bluefors, lambda bf: bf.get_temperature(2), 'K'],
             'T_Still': [ bluefors, lambda bf: bf.get_temperature(5), 'K'],
               'T_MXC': [ bluefors, lambda bf: bf.get_temperature(6), 'K']
}

# Create a new experiment instance where the dictionaries are applied
expr = exp3(control_instr_dict, 
              vna_control_dict, 
            readout_instr_dict, 
         readout_bluefors_dict)

# Constants for frequency conversions
GHz = 1e9
MHz = 1e6
kHz = 1e3

# Function to get the temperatures from the BlueFors fridge
def get_temps():
    temp_data = {}
    for label, (device, func, units) in readout_bluefors_dict.items():
        val = np.around(func(device), 2)
        temp_data[label] = val
        print(f"{label}: {val} {units}")
    return temp_data

Attributes    Values
------------  ------------------
Vlch          0 V
Vlgt          0 V
Vrgt          0 V
Vsign         10 mVpp
Fsign         3 MHz
Vrgd          0 V
Vlres         0 V
Vmres         0 V
Vfila         10 mVpp
Ffila         10 Hz
freq_range    (3, 6) GHz
power         -30 dBm
elec_delay    10 ns
set_vna_meas  ('meas', 'S21', 1)
format        MLOG
output        0
sweep_pts     1e+04 pts
getf_range    (3, 6) GHz
set_bwidth    1 kHz
num_avg       1 avgs
avg_state     OFF
T_50K         51.3 K
T_4K          3.48 K
T_Still       636 mK
T_MXC         36.4 mK


## **Experiment Implementation**

Here is where the values for the specific experiment can be entered.  Default values for when this notebook is initialized are set in the dictionaries above.  However, when these values are changed anywhere else in the notebook, they will remain at that changed value until changed again or the notebook is restarted.  Values do not return to the dictionary defaults on their own.

In [3]:
# Transport signal generator
gen_out = 'off'
gen_sign.output = gen_out

expr.Vsign = 0.01
expr.Fsign = 500*kHz

expr.Vlch  = 0.00
expr.Vrgd  = 0.00
expr.Vlgt  = 0.00
expr.Vrgt  = 0.00
expr.Vlres = 0.00
expr.Vmres = 0.00

expr.set_vna_meas = ('TEST_2', 'S21', 1)
expr.format = 'MLOG'
expr.output = False
expr.power = 0
expr.sweep_pts = 2001
expr.elec_delay = 54.0e-9
expr.freq_range = (4.4*GHz, 4.6*GHz) 
expr.set_bwidth = 0.5e3
expr.num_avg = 1
expr.avg_state = 'OFF'

expr.comment1 = f'VNA Lines = QI,X \ Line Attenutation = 60dBm'
#exp.comment2 = f'vac -20dB \ (RHS) \ Vac {genout} \ BNC ({str(sigout)}) att -40dBm'

expr.table()


Attributes    Values
------------  --------------------------------------------
Vlch          0 V
Vlgt          0 V
Vrgt          0 V
Vsign         10 mVpp
Fsign         500 kHz
Vrgd          0 V
Vlres         0 V
Vmres         0 V
Vfila         10 mVpp
Ffila         10 Hz
freq_range    (4.4, 4.6) GHz
power         0 dBm
elec_delay    54 ns
set_vna_meas  ('TEST_2', 'S21', 1)
format        MLOG
output        0
sweep_pts     2e+03 pts
getf_range    (3, 6) GHz
set_bwidth    500 Hz
num_avg       1 avgs
avg_state     OFF
T_50K         51.3 K
T_4K          3.48 K
T_Still       636 mK
T_MXC         36.4 mK
comment1      VNA Lines = QI,X \ Line Attenutation = 60dBm


In [4]:

expr.sweep_params(
    var     =   ['freq_range'],
    s1      =   4.4*GHz, 
    s2      =   4.6*GHz,   
    num     =   2001,
    scale   =   'linear',
    offset  =   [0]
)

expr.step_params(
    var     =   [],
    s1      =   0.001, 
    s2      =   0.002,   
    num     =   3,
    scale   =   'linear',
    offset  =   [0]
)


# Set the experiment name
exp_name = 'VNA_1D'

# Initialize the VNA for 1D VNA measurements
expr.init_reads(printing=False);

# Run the VNA experiment
expr.run_experiment(exp_name, savedata=True);

Non-index columns: ['empty step', 'freq_range', 'vna_meas', 'vna_freq']


Saving to Database: 100%|██████████| 2001/2001 [00:11<00:00, 167.18it/s]

Data saved to database
